<a href="https://colab.research.google.com/github/julianaschoener/fringe-ir-system/blob/main/Milica_dataset_and_inverted_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install beautifulsoup4 requests

import requests
from bs4 import BeautifulSoup
import os
import re
import json
from google.colab import files
import pandas as pd
import time
import pickle

In [ ]:

os.makedirs("fringe_episodes_full", exist_ok=True)

urls = [
    #season 1
    ("https://fringe.fandom.com/wiki/Pilot", "https://fringe.fandom.com/wiki/Pilot/transcript", "https://fringe.fandom.com/wiki/Pilot/cast"),
    ("https://fringe.fandom.com/wiki/The_Same_Old_Story", "https://fringe.fandom.com/wiki/The_Same_Old_Story/transcript", "https://fringe.fandom.com/wiki/The_Same_Old_Story/cast"),
    ("https://fringe.fandom.com/wiki/The_Ghost_Network", "https://fringe.fandom.com/wiki/The_Ghost_Network/transcript", "https://fringe.fandom.com/wiki/The_Ghost_Network/cast"),
    ("https://fringe.fandom.com/wiki/The_Arrival", "https://fringe.fandom.com/wiki/The_Arrival/transcript", "https://fringe.fandom.com/wiki/The_Arrival/cast"),
    ("https://fringe.fandom.com/wiki/Power_Hungry", "https://fringe.fandom.com/wiki/Power_Hungry/transcript", "https://fringe.fandom.com/wiki/Power_Hungry/cast"),
    ("https://fringe.fandom.com/wiki/The_Cure", "https://fringe.fandom.com/wiki/The_Cure/transcript", "https://fringe.fandom.com/wiki/The_Cure/cast"),
    ("https://fringe.fandom.com/wiki/In_Which_We_Meet_Mr._Jones", "https://fringe.fandom.com/wiki/In_Which_We_Meet_Mr._Jones/transcript", "https://fringe.fandom.com/wiki/In_Which_We_Meet_Mr._Jones/cast"),
    ("https://fringe.fandom.com/wiki/The_Equation", "https://fringe.fandom.com/wiki/The_Equation/transcript", "https://fringe.fandom.com/wiki/The_Equation/cast"),
    ("https://fringe.fandom.com/wiki/The_Dreamscape", "https://fringe.fandom.com/wiki/The_Dreamscape/transcript", "https://fringe.fandom.com/wiki/The_Dreamscape/cast"),
    ("https://fringe.fandom.com/wiki/Safe", "https://fringe.fandom.com/wiki/Safe/transcript", "https://fringe.fandom.com/wiki/Safe/cast"),
    ("https://fringe.fandom.com/wiki/Bound", "https://fringe.fandom.com/wiki/Bound/transcript", "https://fringe.fandom.com/wiki/Bound/cast"),
    ("https://fringe.fandom.com/wiki/The_No-Brainer", "https://fringe.fandom.com/wiki/The_No-Brainer/transcript", "https://fringe.fandom.com/wiki/The_No-Brainer/cast"),
    ("https://fringe.fandom.com/wiki/The_Transformation", "https://fringe.fandom.com/wiki/The_Transformation/transcript", "https://fringe.fandom.com/wiki/The_Transformation/cast"),
    ("https://fringe.fandom.com/wiki/Ability", "https://fringe.fandom.com/wiki/Ability/transcript", "https://fringe.fandom.com/wiki/Ability/cast"),
    ("https://fringe.fandom.com/wiki/Inner_Child", "https://fringe.fandom.com/wiki/Inner_Child/transcript", "https://fringe.fandom.com/wiki/Inner_Child/cast"),
    ("https://fringe.fandom.com/wiki/Unleashed", "https://fringe.fandom.com/wiki/Unleashed/transcript", "https://fringe.fandom.com/wiki/Unleashed/cast"),
    ("https://fringe.fandom.com/wiki/Bad_Dreams", "https://fringe.fandom.com/wiki/Bad_Dreams/transcript", "https://fringe.fandom.com/wiki/Bad_Dreams/cast"),
    ("https://fringe.fandom.com/wiki/Midnight", "https://fringe.fandom.com/wiki/Midnight/transcript", "https://fringe.fandom.com/wiki/Midnight/cast"),
    ("https://fringe.fandom.com/wiki/The_Road_Not_Taken", "https://fringe.fandom.com/wiki/The_Road_Not_Taken/transcript", "https://fringe.fandom.com/wiki/The_Road_Not_Taken/cast"),
    ("https://fringe.fandom.com/wiki/There%27s_More_Than_One_of_Everything", "https://fringe.fandom.com/wiki/There%27s_More_Than_One_of_Everything/transcript", "https://fringe.fandom.com/wiki/There%27s_More_Than_One_of_Everything/cast"),
    #season 2
    ("https://fringe.fandom.com/wiki/A_New_Day_In_The_Old_Town", "https://fringe.fandom.com/wiki/A_New_Day_In_The_Old_Town/transcript", "https://fringe.fandom.com/wiki/A_New_Day_In_The_Old_Town/cast"),
    ("https://fringe.fandom.com/wiki/Night_Of_Desirable_Objects", "https://fringe.fandom.com/wiki/Night_Of_Desirable_Objects/transcript", "https://fringe.fandom.com/wiki/Night_Of_Desirable_Objects/cast"),
    ("https://fringe.fandom.com/wiki/Fracture", "https://fringe.fandom.com/wiki/Fracture/transcript", "https://fringe.fandom.com/wiki/Fracture/cast"),
    ("https://fringe.fandom.com/wiki/Momentum_Deferred", "https://fringe.fandom.com/wiki/Momentum_Deferred/transcript", "https://fringe.fandom.com/wiki/Momentum_Deferred/cast"),
    ("https://fringe.fandom.com/wiki/Dream_Logic", "https://fringe.fandom.com/wiki/Dream_Logic/transcript", "https://fringe.fandom.com/wiki/Dream_Logic/cast"),
    ("https://fringe.fandom.com/wiki/Earthling", "https://fringe.fandom.com/wiki/Earthling", "https://fringe.fandom.com/wiki/Earthling/cast"),
    ("https://fringe.fandom.com/wiki/Of_Human_Action", "https://fringe.fandom.com/wiki/Of_Human_Action/transcript", "https://fringe.fandom.com/wiki/Of_Human_Action/cast"),
    ("https://fringe.fandom.com/wiki/August_(episode)", "https://fringe.fandom.com/wiki/August_(episode)/transcript", "https://fringe.fandom.com/wiki/August_(episode)/cast"),
    ("https://fringe.fandom.com/wiki/Snakehead", "https://fringe.fandom.com/wiki/Snakehead/transcript", "https://fringe.fandom.com/wiki/Snakehead/cast"),
    ("https://fringe.fandom.com/wiki/Grey_Matters", "https://fringe.fandom.com/wiki/Grey_Matters/transcript", "https://fringe.fandom.com/wiki/Grey_Matters/cast"),
    ("https://fringe.fandom.com/wiki/Unearthed", "https://fringe.fandom.com/wiki/Unearthed/transcript", "https://fringe.fandom.com/wiki/Unearthed/cast"),
    ("https://fringe.fandom.com/wiki/Johari_Window", "https://fringe.fandom.com/wiki/Johari_Window/transcript", "https://fringe.fandom.com/wiki/Johari_Window/cast"),
    ("https://fringe.fandom.com/wiki/What_Lies_Below", "https://fringe.fandom.com/wiki/What_Lies_Below/transcript", "https://fringe.fandom.com/wiki/What_Lies_Below/cast"),
    ("https://fringe.fandom.com/wiki/The_Bishop_Revival", "https://fringe.fandom.com/wiki/The_Bishop_Revival/transcript", "https://fringe.fandom.com/wiki/The_Bishop_Revival/cast"),
    ("https://fringe.fandom.com/wiki/Jacksonville", "https://fringe.fandom.com/wiki/Jacksonville/transcript", "https://fringe.fandom.com/wiki/Jacksonville/cast"),
    ("https://fringe.fandom.com/wiki/Peter_(episode)", "https://fringe.fandom.com/wiki/Peter_(episode)/transcript", "https://fringe.fandom.com/wiki/Peter_(episode)/cast"),
    ("https://fringe.fandom.com/wiki/Olivia._In_the_Lab._With_the_Revolver", "https://fringe.fandom.com/wiki/Olivia._In_the_Lab._With_the_Revolver/transcript", "https://fringe.fandom.com/wiki/Olivia._In_the_Lab._With_the_Revolver/cast"),
    ("https://fringe.fandom.com/wiki/White_Tulip", "https://fringe.fandom.com/wiki/White_Tulip/transcript", "https://fringe.fandom.com/wiki/White_Tulip/cast"),
    ("https://fringe.fandom.com/wiki/The_Man_from_the_Other_Side", "https://fringe.fandom.com/wiki/The_Man_from_the_Other_Side/transcript", "https://fringe.fandom.com/wiki/The_Man_from_the_Other_Side/cast"),
    ("https://fringe.fandom.com/wiki/Brown_Betty", "https://fringe.fandom.com/wiki/Brown_Betty/transcript", "https://fringe.fandom.com/wiki/Brown_Betty/cast"),
    ("https://fringe.fandom.com/wiki/Northwest_Passage", "https://fringe.fandom.com/wiki/Northwest_Passage/transcript", "https://fringe.fandom.com/wiki/Northwest_Passage/cast"),
    ("https://fringe.fandom.com/wiki/Over_There:_Part_1", "https://fringe.fandom.com/wiki/Over_There:_Part_1/transcript", "https://fringe.fandom.com/wiki/Over_There:_Part_1/cast"),
    ("https://fringe.fandom.com/wiki/Over_There:_Part_2", "https://fringe.fandom.com/wiki/Over_There:_Part_2/transcript", "https://fringe.fandom.com/wiki/Over_There:_Part_2/cast"),
    #season 3
    ("https://fringe.fandom.com/wiki/Olivia_(episode)", "https://fringe.fandom.com/wiki/Olivia_(episode)/transcript", "https://fringe.fandom.com/wiki/Olivia_(episode)/cast"),
    ("https://fringe.fandom.com/wiki/The_Box", "https://fringe.fandom.com/wiki/The_Box/transcript", "https://fringe.fandom.com/wiki/The_Box/cast"),
    ("https://fringe.fandom.com/wiki/The_Plateau", "https://fringe.fandom.com/wiki/The_Plateau/transcript", "https://fringe.fandom.com/wiki/The_Plateau/cast"),
    ("https://fringe.fandom.com/wiki/Do_Shapeshifters_Dream_Of_Electric_Sheep%3F", "https://fringe.fandom.com/wiki/Do_Shapeshifters_Dream_Of_Electric_Sheep%3F/transcript", "https://fringe.fandom.com/wiki/Do_Shapeshifters_Dream_Of_Electric_Sheep%3F/cast"),
    ("https://fringe.fandom.com/wiki/Amber_31422", "https://fringe.fandom.com/wiki/Amber_31422/transcript", "https://fringe.fandom.com/wiki/Amber_31422/cast"),
    ("https://fringe.fandom.com/wiki/6955_kHz", "https://fringe.fandom.com/wiki/6955_kHz/transcript", "https://fringe.fandom.com/wiki/6955_kHz/cast"),
    ("https://fringe.fandom.com/wiki/The_Abducted", "https://fringe.fandom.com/wiki/The_Abducted/transcript", "https://fringe.fandom.com/wiki/The_Abducted/cast"),
    ("https://fringe.fandom.com/wiki/Entrada", "https://fringe.fandom.com/wiki/Entrada/transcript", "https://fringe.fandom.com/wiki/Entrada/cast"),
    ("https://fringe.fandom.com/wiki/Marionette", "https://fringe.fandom.com/wiki/Marionette/transcript", "https://fringe.fandom.com/wiki/Marionette/cast"),
    ("https://fringe.fandom.com/wiki/The_Firefly", "https://fringe.fandom.com/wiki/The_Firefly/transcript", "https://fringe.fandom.com/wiki/The_Firefly/cast"),
    ("https://fringe.fandom.com/wiki/Reciprocity", "https://fringe.fandom.com/wiki/Reciprocity/transcript", "https://fringe.fandom.com/wiki/Reciprocity/cast"),
    ("https://fringe.fandom.com/wiki/Concentrate_and_Ask_Again", "https://fringe.fandom.com/wiki/Concentrate_and_Ask_Again/transcript", "https://fringe.fandom.com/wiki/Concentrate_and_Ask_Again/cast"),
    ("https://fringe.fandom.com/wiki/Immortality", "https://fringe.fandom.com/wiki/Immortality/transcript", "https://fringe.fandom.com/wiki/Immortality/cast"),
    ("https://fringe.fandom.com/wiki/6B", "https://fringe.fandom.com/wiki/6B/transcript", "https://fringe.fandom.com/wiki/6B/cast"),
    ("https://fringe.fandom.com/wiki/Subject_13", "https://fringe.fandom.com/wiki/Subject_13/transcript", "https://fringe.fandom.com/wiki/Subject_13/cast"),
    ("https://fringe.fandom.com/wiki/Os", "https://fringe.fandom.com/wiki/Os/transcript", "https://fringe.fandom.com/wiki/Os/cast"),
    ("https://fringe.fandom.com/wiki/Stowaway", "https://fringe.fandom.com/wiki/Stowaway/transcript", "https://fringe.fandom.com/wiki/Stowaway/cast"),
    ("https://fringe.fandom.com/wiki/Bloodline", "https://fringe.fandom.com/wiki/Bloodline/transcript", "https://fringe.fandom.com/wiki/Bloodline/cast"),
    ("https://fringe.fandom.com/wiki/Lysergic_Acid_Diethylamide", "https://fringe.fandom.com/wiki/Lysergic_Acid_Diethylamide", "https://fringe.fandom.com/wiki/Lysergic_Acid_Diethylamide/cast"),
    ("https://fringe.fandom.com/wiki/6:02_AM_EST", "https://fringe.fandom.com/wiki/6:02_AM_EST/transcript", "https://fringe.fandom.com/wiki/6:02_AM_EST/cast"),
    ("https://fringe.fandom.com/wiki/The_Last_Sam_Weiss", "https://fringe.fandom.com/wiki/The_Last_Sam_Weiss/transcript", "https://fringe.fandom.com/wiki/The_Last_Sam_Weiss/cast"),
    ("https://fringe.fandom.com/wiki/The_Day_We_Died", "https://fringe.fandom.com/wiki/The_Day_We_Died/transcript", "https://fringe.fandom.com/wiki/The_Day_We_Died/cast"),
    #season 4
    ("https://fringe.fandom.com/wiki/Neither_Here_nor_There", "https://fringe.fandom.com/wiki/Neither_Here_nor_There/transcript", "https://fringe.fandom.com/wiki/Neither_Here_nor_There/cast"),
    ("https://fringe.fandom.com/wiki/One_Night_in_October", "https://fringe.fandom.com/wiki/One_Night_in_October/transcript", "https://fringe.fandom.com/wiki/One_Night_in_October/cast"),
    ("https://fringe.fandom.com/wiki/Alone_in_the_World", "https://fringe.fandom.com/wiki/Alone_in_the_World/transcript", "https://fringe.fandom.com/wiki/Alone_in_the_World/cast"),
    ("https://fringe.fandom.com/wiki/Subject_9", "https://fringe.fandom.com/wiki/Subject_9/transcript", "https://fringe.fandom.com/wiki/Subject_9/cast"),
    ("https://fringe.fandom.com/wiki/Novation", "https://fringe.fandom.com/wiki/Novation/transcript", "https://fringe.fandom.com/wiki/Novation/cast"),
    ("https://fringe.fandom.com/wiki/And_Those_We%27ve_Left_Behind", "https://fringe.fandom.com/wiki/And_Those_We%27ve_Left_Behind/transcript", "https://fringe.fandom.com/wiki/And_Those_We%27ve_Left_Behind/cast"),
    ("https://fringe.fandom.com/wiki/Wallflower", "https://fringe.fandom.com/wiki/Wallflower/transcript", "https://fringe.fandom.com/wiki/Wallflower/cast"),
    ("https://fringe.fandom.com/wiki/Back_to_Where_You%27ve_Never_Been", "https://fringe.fandom.com/wiki/Back_to_Where_You%27ve_Never_Been/transcript", "https://fringe.fandom.com/wiki/Back_to_Where_You%27ve_Never_Been/cast"),
    ("https://fringe.fandom.com/wiki/Enemy_of_My_Enemy", "https://fringe.fandom.com/wiki/Enemy_of_My_Enemy/transcript", "https://fringe.fandom.com/wiki/Enemy_of_My_Enemy/cast"),
    ("https://fringe.fandom.com/wiki/Forced_Perspective", "https://fringe.fandom.com/wiki/Forced_Perspective/transcript", "https://fringe.fandom.com/wiki/Forced_Perspective/cast"),
    ("https://fringe.fandom.com/wiki/Making_Angels", "https://fringe.fandom.com/wiki/Making_Angels/transcript", "https://fringe.fandom.com/wiki/Making_Angels/cast"),
    ("https://fringe.fandom.com/wiki/Welcome_to_Westfield", "https://fringe.fandom.com/wiki/Welcome_to_Westfield/transcript", "https://fringe.fandom.com/wiki/Welcome_to_Westfield/cast"),
    ("https://fringe.fandom.com/wiki/A_Better_Human_Being", "https://fringe.fandom.com/wiki/A_Better_Human_Being/transcript", "https://fringe.fandom.com/wiki/A_Better_Human_Being/cast"),
    ("https://fringe.fandom.com/wiki/The_End_of_All_Things", "https://fringe.fandom.com/wiki/The_End_of_All_Things/transcript", "https://fringe.fandom.com/wiki/The_End_of_All_Things/cast"),
    ("https://fringe.fandom.com/wiki/A_Short_Story_About_Love", "https://fringe.fandom.com/wiki/A_Short_Story_About_Love/transcript", "https://fringe.fandom.com/wiki/A_Short_Story_About_Love/cast"),
    ("https://fringe.fandom.com/wiki/Nothing_As_It_Seems", "https://fringe.fandom.com/wiki/Nothing_As_It_Seems/transcript", "https://fringe.fandom.com/wiki/Nothing_As_It_Seems/cast"),
    ("https://fringe.fandom.com/wiki/Everything_In_Its_Right_Place", "https://fringe.fandom.com/wiki/Everything_In_Its_Right_Place/transcript", "https://fringe.fandom.com/wiki/Everything_In_Its_Right_Place/cast"),
    ("https://fringe.fandom.com/wiki/The_Consultant", "https://fringe.fandom.com/wiki/The_Consultant/transcript", "https://fringe.fandom.com/wiki/The_Consultant/cast"),
    ("https://fringe.fandom.com/wiki/Letters_of_Transit", "https://fringe.fandom.com/wiki/Letters_of_Transit/transcript", "https://fringe.fandom.com/wiki/Letters_of_Transit/cast"),
    ("https://fringe.fandom.com/wiki/Worlds_Apart", "https://fringe.fandom.com/wiki/Worlds_Apart/transcript", "https://fringe.fandom.com/wiki/Worlds_Apart/cast"),
    ("https://fringe.fandom.com/wiki/Brave_New_World:_Part_1", "https://fringe.fandom.com/wiki/Brave_New_World:_Part_1/transcript", "https://fringe.fandom.com/wiki/Brave_New_World:_Part_1/cast"),
    ("https://fringe.fandom.com/wiki/Brave_New_World:_Part_2", "https://fringe.fandom.com/wiki/Brave_New_World:_Part_2/transcript", "https://fringe.fandom.com/wiki/Brave_New_World:_Part_2/cast"),
    #season 5
        ("https://fringe.fandom.com/wiki/Transilience_Thought_Unifier_Model-11", "https://fringe.fandom.com/wiki/Transilience_Thought_Unifier_Model-11/transcript", "https://fringe.fandom.com/wiki/Transilience_Thought_Unifier_Model-11/cast"),
    ("https://fringe.fandom.com/wiki/In_Absentia", "https://fringe.fandom.com/wiki/In_Absentia/transcript", "https://fringe.fandom.com/wiki/In_Absentia/cast"),
    ("https://fringe.fandom.com/wiki/The_Recordist", "https://fringe.fandom.com/wiki/The_Recordist/transcript", "https://fringe.fandom.com/wiki/The_Recordist/cast"),
    ("https://fringe.fandom.com/wiki/The_Bullet_That_Saved_The_World", "https://fringe.fandom.com/wiki/The_Bullet_That_Saved_The_World/transcript", "https://fringe.fandom.com/wiki/The_Bullet_That_Saved_The_World/cast"),
    ("https://fringe.fandom.com/wiki/An_Origin_Story", "https://fringe.fandom.com/wiki/An_Origin_Story/transcript", "https://fringe.fandom.com/wiki/An_Origin_Story/cast"),
    ("https://fringe.fandom.com/wiki/Through_the_Looking_Glass_and_What_Walter_Found_There", "https://fringe.fandom.com/wiki/Through_the_Looking_Glass_and_What_Walter_Found_There/transcript", "https://fringe.fandom.com/wiki/Through_the_Looking_Glass_and_What_Walter_Found_There/cast"),
    ("https://fringe.fandom.com/wiki/Five-Twenty-Ten", "https://fringe.fandom.com/wiki/Five-Twenty-Ten/transcript", "https://fringe.fandom.com/wiki/Five-Twenty-Ten/cast"),
    ("https://fringe.fandom.com/wiki/The_Human_Kind", "https://fringe.fandom.com/wiki/The_Human_Kind/transcript", "https://fringe.fandom.com/wiki/The_Human_Kind/cast"),
    ("https://fringe.fandom.com/wiki/Black_Blotter", "https://fringe.fandom.com/wiki/Black_Blotter/transcript", "https://fringe.fandom.com/wiki/Black_Blotter/cast"),
    ("https://fringe.fandom.com/wiki/Anomaly_XB-6783746", "https://fringe.fandom.com/wiki/Anomaly_XB-6783746/transcript", "https://fringe.fandom.com/wiki/Anomaly_XB-6783746/cast"),
    ("https://fringe.fandom.com/wiki/The_Boy_Must_Live", "https://fringe.fandom.com/wiki/The_Boy_Must_Live/transcript", "https://fringe.fandom.com/wiki/The_Boy_Must_Live/cast"),
    ("https://fringe.fandom.com/wiki/Liberty", "https://fringe.fandom.com/wiki/Liberty/transcript", "https://fringe.fandom.com/wiki/Liberty/cast"),
    ("https://fringe.fandom.com/wiki/An_Enemy_of_Fate", "https://fringe.fandom.com/wiki/An_Enemy_of_Fate/transcript", "https://fringe.fandom.com/wiki/An_Enemy_of_Fate/cast")
]




In [ ]:
from sre_constants import IN_IGNORE
def clean_text(text):
    return ' '.join(text.strip().split())

def extract_season_episode(page_text):
    season = "Unknown"
    episode = "Unknown"

    match = re.search(r"Season\s+(\d+)[,\s]+Episode\s+(\d+)", page_text, re.IGNORECASE)
    if match:
        season = match.group(1)
        episode = match.group(2)
    else:
        match = re.search(r"S(\d+)E(\d+)", page_text, re.IGNORECASE)
        if match:
            season = match.group(1)
            episode = match.group(2)

    return season, episode

def extract_writers_directors(page_text):
    writers = "Unknown"
    directors = "Unknown"

    writer_match = re.search(r'Written by[:\s]+([^\n]+)', page_text, re.IGNORECASE)
    director_match = re.search(r'Directed by[:\s]+([^\n]+)', page_text, re.IGNORECASE)

    if writer_match:
        writers = clean_text(writer_match.group(1))
    if director_match:
        directors = clean_text(director_match.group(1))

    return writers, directors

def extract_cast(page_text):
    cast_list = []

    matches = re.findall(r'([A-Z][a-z]+\s+[A-Z][a-z]+)\s+as\s+([A-Z][a-zA-Z\s]+)', page_text)
    for actor, character in matches:
        cast_list.append(f"{actor} as {character}")

    return list(set(cast_list))

def extract_script(transcript_soup):
    script_text = ""
    content_div = transcript_soup.find("div", {"class": "mw-parser-output"})
    if content_div:
        paragraphs = content_div.find_all(["p", "ul", "ol"])
        for tag in paragraphs:
            script_text += clean_text(tag.text) + "\n"
    return script_text.strip()

dataset = []
episode_id = 1
for episode_url, transcript_url, cast_url in urls:
    print(f"Scraping {episode_url}...")
    try:
        ep_page = requests.get(episode_url)
        tr_page = requests.get(transcript_url)
        cast_page = requests.get(cast_url)

        episode_soup = BeautifulSoup(ep_page.content, "html.parser")
        transcript_soup = BeautifulSoup(tr_page.content, "html.parser")
        cast_soup = BeautifulSoup(cast_page.content, "html.parser")

        ep_text = episode_soup.get_text(separator='\n')
        cast_text = cast_soup.get_text(separator='\n')

        writers, directors = extract_writers_directors(ep_text)
        season, episode = extract_season_episode(ep_text)

        cast = extract_cast(cast_text)

        title_tag = episode_soup.find("h1", id="firstHeading")
        title = clean_text(title_tag.text) if title_tag else "Unknown Title"

        script = extract_script(transcript_soup)

        dataset.append({
            "id": episode_id,
            "Title": title,
            "Season": season,
            "Episode": episode,
            "Writers": writers,
            "Directors": directors,
            "Cast": cast,
            "Script": script
        })
        episode_id += 1
        time.sleep(1)

    except Exception as e:
        print(f"Failed scraping {episode_url}: {e}")

df = pd.DataFrame(dataset)
df.to_csv("fringe_dataset.csv", index=False)
print("Saved dataset to fringe_dataset.csv ✅")
df.to_json("fringe_dataset.json", orient="records", indent=2)
print("Saved to fringe_episodes.json ✅")

#from google.colab import files
#files.download("fringe_dataset.csv")
#files.download("fringe_dataset.json")
#print("Files ready for download!")

Scraping https://fringe.fandom.com/wiki/Pilot...
Scraping https://fringe.fandom.com/wiki/The_Same_Old_Story...
Scraping https://fringe.fandom.com/wiki/The_Ghost_Network...
Scraping https://fringe.fandom.com/wiki/The_Arrival...
Scraping https://fringe.fandom.com/wiki/Power_Hungry...
Scraping https://fringe.fandom.com/wiki/The_Cure...
Scraping https://fringe.fandom.com/wiki/In_Which_We_Meet_Mr._Jones...
Scraping https://fringe.fandom.com/wiki/The_Equation...
Scraping https://fringe.fandom.com/wiki/The_Dreamscape...
Scraping https://fringe.fandom.com/wiki/Safe...
Scraping https://fringe.fandom.com/wiki/Bound...
Scraping https://fringe.fandom.com/wiki/The_No-Brainer...
Scraping https://fringe.fandom.com/wiki/The_Transformation...
Scraping https://fringe.fandom.com/wiki/Ability...
Scraping https://fringe.fandom.com/wiki/Inner_Child...
Scraping https://fringe.fandom.com/wiki/Unleashed...
Scraping https://fringe.fandom.com/wiki/Bad_Dreams...
Scraping https://fringe.fandom.com/wiki/Midnight...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Files ready for download!


In [ ]:
with open('fringe_dataset.pkl', 'wb') as f:
    pickle.dump(dataset, f)


In [ ]:
inverted_index = {}

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    words = text.split()
    return words

for episode in dataset:
    doc_id = episode['id']
    script = episode['Script']
    tokens = preprocess(script)

    for token in tokens:
        if token not in inverted_index:
            inverted_index[token] = set()
        inverted_index[token].add(doc_id)

with open('inverted_index.pkl', 'wb') as f:
    pickle.dump(inverted_index, f)

In [ ]:
with open('inverted_index.pkl', 'rb') as f:
    inverted_index = pickle.load(f)


print(inverted_index.get("amber", []))
print(inverted_index.get("pod", []))
print(inverted_index.get("tulip", []))

{2, 3, 23, 33, 43, 48, 57, 63, 65, 67, 73, 84, 88, 89, 90, 91, 92, 93, 94, 98, 99}
{24, 46}
{38, 58, 98, 63}


In [ ]:
def search(query):
    query_words = preprocess(query)
    result_sets = []

    for word in query_words:
        if word in inverted_index:
            result_sets.append(set(inverted_index[word]))
        else:
            result_sets.append(set())

    if result_sets:
        result_ids = set.intersection(*result_sets)
    else:
        result_ids = set()

    return sorted(result_ids)

query = "why does september save peter"
results = search(query)
print(f"🔍 Query: '{query}' -> Matching episode IDs: {results}")


🔍 Query: 'why does september save peter' -> Matching episode IDs: [35, 45, 76, 84, 87, 88, 90, 91, 97, 99]
